<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
import os
import datetime as dt
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

In [ ]:
# Stock tickers - Insert here# Real Estate Invesments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IBOV + 25 assets in alphabetical order to easly adjust google docs spreadsheet
tickers = ['^BVSP','ASAI3.SA','AURE3.SA','B3SA3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CGAS3.SA','CPFE3.SA','CSAN3.SA','EGIE3.SA','ELET6.SA','IVVB11.SA','JALL3.SA','JHSF3.SA','KLBN11.SA','LEVE3.SA','LREN3.SA','ODPV3.SA','PETR4.SA','PRIO3.SA','RAIZ4.SA','STBP3.SA','TAEE11.SA','VALE3.SA','VIVT3.SA']

In [ ]:
# Real Estate Invesments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IFIX + 20 assets in alphabetical order to easly adjust google docs spreadsheet
# IFIX here only to a space in dataframe, yahoo finance return IFIX.SA maximum 5 days. IFIX should be calculated downloading an Investing.com csv file history and it must be uploaded in personal Google Drive. This code read it.
tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MALL11.SA','MFII11.SA','SADI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']

In [ ]:
# US Stocks tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 20 assets in alphabetical order to easly adjust google docs spreadsheet
tickersus = ['^GSPC','USDBRL=X','AAPL','AIG','BAC','BHP','DHI','EXC','KMB','KO','LOPE','LYB','MGA','MSFT','MSTR','NVDA','TGT','TMUS','UPS','UNH','X','XOM']

In [ ]:
# US ETFs tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 10 assets in alphabetical order to easly adjust google docs spreadsheet
tickereus = ['^GSPC','USDBRL=X','FBTC','JEPI','HACK', 'IVV','SCHD','SOXX','SPY','TLT','SMH','TFLO']

In [ ]:
# Portfolio tickers - INSERT OR ADJUST HERE
tickerport = ['^BVSP','USDBRL=X','BBAS3.SA','CPFE3.SA','ELET6.SA','FBTC','HSML11.SA','HTMX11.SA','IVVB11.SA','PETR4.SA','SADI11.SA','SCHD','TFLO','TGAR11.SA','TRXF11.SA','VALE3.SA']
# Portfolio tickers weight - INSERT OR ADJUST HERE IN THE SAME ORDER!
weightport = [0.0, 0.0, 5.885, 3.688, 5.110, 3.314, 2.776, 6.154, 9.425, 7.494, 7.068, 32.646, 4.042, 4.111, 5.118, 3.169]
# Portfolio tickers expected returns - INSERT OR ADJUST HERE IN THE SAME ORDER!
# IBOV (^BVSP) MUST always be zero!
expretport = [0.0, 0.0, 27.3, 15.0, 1.5, 30.0, 12.0, 32.4, 17.0, 15.0, 20.8, 17.0, 10.0, 18.2, 17.6, 10.9]

In [ ]:
# Risk free rate in percentage - INSERT OR ADJUST HERE
riskfree = 13.25

In [ ]:
# Portfolio dataframe creation
portfolio = pd.DataFrame({'Ticker': tickerport, 'W': weightport, 'RetE%':expretport})

In [ ]:
# Excluding .SA, renaming ^BVSP to IBOV and USDBRL=X to USDBRL
portfolio['Ticker'] = portfolio['Ticker'].str.replace('.SA', '', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('^BVSP', 'IBOV', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('USDBRL=X', 'USDBRL', regex=False)
# display(portfolio)

In [ ]:
# Load tickers history prices in a dataframe considering a certain period of time - ADJUST HERE, default 1 year (1y)
# Sometimes some ticker has problems in yahoo finance. If it happens, close the session and try again. Or change the ticker, because problem in one ticker will cause problem in all code running.
# Check success download completed to all dataframes, otherwise the code will broke in next lines.
dfs = yf.download(tickers, period='1y')['Close']
dfr = yf.download(tickerr, period='1y')['Close']
dfsus = yf.download(tickersus, period='1y')['Close']
dfeus = yf.download(tickereus, period='1y')['Close']
dfport = yf.download(tickerport, period='1y')['Close']
# Remove timezone from index
dfs.index = pd.to_datetime(dfs.index).tz_localize(None)
dfr.index = pd.to_datetime(dfr.index).tz_localize(None)
dfsus.index = pd.to_datetime(dfsus.index).tz_localize(None)
dfeus.index = pd.to_datetime(dfeus.index).tz_localize(None)
dfport.index = pd.to_datetime(dfport.index).tz_localize(None)
# display(dfs)
# display(dfr)
# display(dfeus)
# display(dfport)

[*********************100%***********************]  26 of 26 completed
[*********************100%***********************]  21 of 21 completed
[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  16 of 16 completed


In [ ]:
# Convert US assets in BRL values, it should be adjusted according the assets in portfolio - ADJUST HERE
dfport['SCHD'] = dfport['SCHD'] * dfport['USDBRL=X']
dfport['TFLO'] = dfport['TFLO'] * dfport['USDBRL=X']
dfport['FBTC'] = dfport['FBTC'] * dfport['USDBRL=X']
# display(dfport)

In [ ]:
# IFIX historic series from Investing.com to be appended in real state dataframe dfr - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data
# Download the file from site and copy to your google drive. Rename de file as history.csv. Adjust the path below in " ifixfile = .... " command line according your file location.
# Google Drive mounth
drive.mount('/content/drive', force_remount=True)
# File path on Google Drive - Download the file and upload to Financas folder in Google Drive. Rename the path according file name uploaded.
ifixfile = '/content/drive/MyDrive/Financas/history.csv'
# File csv to dataframe converting quote to float
ifix = pd.read_csv(ifixfile, thousands = '.', decimal = ',', dtype = {'Último':np.float64})
# Excluding and rename columns
ifix = ifix.drop(columns=['Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'])
ifix = ifix.rename(columns={'Data': 'Date', 'Último': 'IFIX.SA'})
# Date format in Date column
ifix['Date'] = pd.to_datetime(ifix['Date'], format='%d%m%Y', errors='coerce')
ifix.set_index('Date', inplace=True)
# Solve eventual duplicated registers, grouped by mean
ifix = ifix.groupby(level=0).mean()
# display(ifix)

Mounted at /content/drive


In [ ]:
# Replace dfr dataframe by ifix values by index key (date)
dfr.update(ifix)
# display(dfr)

In [ ]:
# Excluding .SA, renaming ^BVSP to IBOV, ^GSPC to SP500
dfs.columns = [col.replace('.SA', '') for col in dfs.columns]
dfs.columns = [col.replace('^BVSP', 'IBOV') for col in dfs.columns]
dfr.columns = [col.replace('.SA', '') for col in dfr.columns]
dfsus.columns = [col.replace('^GSPC', 'SP500') for col in dfsus.columns]
dfsus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfsus.columns]
dfeus.columns = [col.replace('^GSPC', 'SP500') for col in dfeus.columns]
dfeus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfeus.columns]
dfport.columns = [col.replace('.SA', '') for col in dfport.columns]
dfport.columns = [col.replace('^BVSP', 'IBOV') for col in dfport.columns]
dfport.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfport.columns]

In [ ]:
# Exclude NaNs, in the first row and in the dfr Date registers that not included in IFIX excel file.
dfs.dropna(inplace=True)
dfr.dropna(inplace=True)
dfsus.dropna(inplace=True)
dfeus.dropna(inplace=True)
dfport.dropna(inplace=True)
# Other conformations such as ascending order and the market indexes in the first column
dfs = dfs[sorted(dfs.columns)]
dfr = dfr[sorted(dfr.columns)]
dfsus = dfsus[sorted(dfsus.columns)]
dfus = dfeus[sorted(dfeus.columns)]
dfport = dfport[sorted(dfport.columns)]
dfs = dfs[['IBOV'] + [col for col in dfs.columns if col != 'IBOV']]
dfr = dfr[['IFIX'] + [col for col in dfr.columns if col != 'IFIX']]
dfsus = dfsus[['SP500', 'USDBRL'] + [col for col in dfsus.columns if col not in ['SP500', 'USDBRL']]]
dfeus = dfeus[['SP500', 'USDBRL'] + [col for col in dfeus.columns if col not in ['SP500', 'USDBRL']]]
dfport = dfport[['IBOV', 'USDBRL'] + [col for col in dfport.columns if col not in ['IBOV', 'USDBRL']]]
# Here dataframes should be ready for calculations. It will be made and uploaded in dfxvar dataframes later.
# display(dfs)
# display(dfr)
# display(dfsus)
# display(dfeus)
# display(dfport)

In [ ]:
# Calculate daily variation
dfsvar = dfs.pct_change()
dfrvar = dfr.pct_change()
dfsusvar = dfsus.pct_change()
dfeusvar = dfeus.pct_change()
dfportvar = dfport.pct_change()
# Excluding MaNs first line
dfsvar.dropna(inplace=True)
dfrvar.dropna(inplace=True)
dfsusvar.dropna(inplace=True)
dfeusvar.dropna(inplace=True)
dfportvar.dropna(inplace=True)
# display(dfsvar)
# display(dfrvar)
# display(dfsusvar)
# display(dfeusvar)
# display(dfportvar)

In [ ]:
# Market Percentage Return calculation and column add in output dataframes
stockvar = pd.DataFrame(dfsvar.mean()*252*100).rename(columns={0: 'Ret%'})
stockvar.index.name = 'Ticker'
stockvar['Ret%'] = stockvar['Ret%'].round(1)
realstatevar = pd.DataFrame(dfrvar.mean()*252*100).rename(columns={0: 'Ret%'})
realstatevar.index.name = 'Ticker'
realstatevar['Ret%'] = realstatevar['Ret%'].round(1)
stockusvar = pd.DataFrame(dfsusvar.mean()*252*100).rename(columns={0: 'Ret%'})
stockusvar.index.name = 'Ticker'
stockusvar['Ret%'] = stockusvar['Ret%'].round(1)
etfusvar = pd.DataFrame(dfeusvar.mean()*252*100).rename(columns={0: 'Ret%'})
etfusvar.index.name = 'Ticker'
etfusvar['Ret%'] = etfusvar['Ret%'].round(1)
portvar = pd.DataFrame(dfportvar.mean()*252*100).rename(columns={0: 'Ret%'})
portvar.index.name = 'Ticker'
portvar['Ret%'] = portvar['Ret%'].round(1)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [ ]:
# Market return variance calculation
vars = dfsvar.var()*252
varr = dfrvar.var()*252
varsus = dfsusvar.var()*252
vareus = dfeusvar.var()*252
varport = dfportvar.var()*252
# display(vars)
# display(varr)
# display(varsus)
# display(vareus)
# display(varport)

In [ ]:
# Market risk calculation, in percentage (%). Add column in output dataframes
stockvar['Risk%'] = dfsvar.std()*np.sqrt(252)*100
stockvar['Risk%'] = stockvar['Risk%'].round(0)
realstatevar['Risk%'] = dfrvar.std()*np.sqrt(252)*100
realstatevar['Risk%'] = realstatevar['Risk%'].round(0)
stockusvar['Risk%'] = dfsusvar.std()*np.sqrt(252)*100
stockusvar['Risk%'] = stockusvar['Risk%'].round(0)
etfusvar['Risk%'] = dfeusvar.std()*np.sqrt(252)*100
etfusvar['Risk%'] = etfusvar['Risk%'].round(0)
portvar['Risk%'] = dfportvar.std()*np.sqrt(252)*100
portvar['Risk%'] = portvar['Risk%'].round(0)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [ ]:
# Covariance calculation
covs = dfsvar.cov()*252
covr = dfrvar.cov()*252
covsus = dfsusvar.cov()*252
coveus = dfeusvar.cov()*252
covport = dfportvar.cov()*252
# display(covs)
# display(covr)
# display(covsus)
# display(coveus)
# display(covport)

In [ ]:
# Beta calculation
betas = covs['IBOV']/vars['IBOV']
betas = betas.round(3)
betas.name = 'Beta'
betar = covr['IFIX']/varr['IFIX']
betar = betar.round(3)
betar.name = 'Beta'
betasus = covsus['SP500']/varsus['SP500']
betasus = betasus.round(3)
betasus.name = 'Beta'
betaeus = coveus['SP500']/vareus['SP500']
betaeus = betaeus.round(3)
betaeus.name = 'Beta'
betaport = covport['IBOV']/varport['IBOV']
betaport = betaport.round(3)
betaport.name = 'Beta'
# display(betas)
# display(betar)
# display(betasus)
# display(betaeus)
# display(betaport)

In [ ]:
# Adding Beta to column output dataframes
stockvar['Beta'] = betas
realstatevar['Beta'] = betar
stockusvar['Beta'] = betasus
etfusvar['Beta'] = betaeus
portvar['Beta'] = betaport
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [ ]:
# Adding Min to column output dataframes
stockvar['Min'] = dfs.min()
stockvar['Min'] = stockvar['Min'].round(2)
realstatevar['Min'] = dfr.min()
realstatevar['Min'] = realstatevar['Min'].round(2)
stockusvar['Min'] = dfsus.min()
stockusvar['Min'] = stockusvar['Min'].round(2)
etfusvar['Min'] = dfeus.min()
etfusvar['Min'] = etfusvar['Min'].round(2)
portvar['Min'] = dfport.min()
portvar['Min'] = portvar['Min'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [ ]:
# Adding Max to column output dataframes
stockvar['Max'] = dfs.max()
stockvar['Max'] = stockvar['Max'].round(2)
realstatevar['Max'] = dfr.max()
realstatevar['Max'] = realstatevar['Max'].round(2)
stockusvar['Max'] = dfsus.max()
stockusvar['Max'] = stockusvar['Max'].round(2)
etfusvar['Max'] = dfeus.max()
etfusvar['Max'] = etfusvar['Max'].round(2)
portvar['Max'] = dfport.max()
portvar['Max'] = portvar['Max'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [ ]:
# Organizing columns order and making Index column as index of dataframes
stockvar = stockvar.reset_index()
realstatevar = realstatevar.reset_index()
stockusvar = stockusvar.reset_index()
etfusvar = etfusvar.reset_index()
portvar = portvar.reset_index()
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [ ]:
# Portfolio dataframe assembling
portfolio = pd.merge(portfolio, portvar, on='Ticker', how='inner')
# portfolio = portfolio[sorted(portfolio.columns)]
# portfolio = portfolio[['IBOV', 'USDBRL'] + [col for col in portfolio.columns if col not in ['IBOV', 'USDBRL']]]
# firstsnames = ['IBOV','USDBRL']
firstsnames = ['IBOV', 'USDBRL']
portfolio.iloc[2:] = portfolio[~portfolio['Ticker'].isin(firstsnames)].sort_values(by='Ticker').values
# display(portfolio)

In [ ]:
# Portfolio Total Return calculation
portfolioretexptotal = (portfolio['W']/100).dot(portfolio['RetE%'])
portfoliorettotal = (portfolio['W']/100).dot(portfolio['Ret%'])
# display(portfoliorettotal)

In [ ]:
# Portolio Total Variance calculation
# It uses LINEAR ALGEBRA, LINES AND COLUMNS MUST BE ALINGNED! CONFIRM IT, check covariance calculations (covport) and portfolio lines. SHOULD BE EQUAL!
# Converts weights to column array
weight = (portfolio['W']/100).to_numpy().reshape(-1, 1) # Converte para array coluna
# Matrix multiplications: Variance = Weight.T x Covariance x Weight. See Portfolio Theory. T means transposed matrix.
# The result should be a dataframe 1x1 with only the value of total portfolio variance
# other line code possible: portvartotal = weight.T @ covport @ weight
portvartotal = (weight.T.dot(covport).dot(weight)).item()
# display(weight)
# display(portvartotal)

In [ ]:
# Total Portfolio Percentage Risk calculation
portfoliorisktotal = (np.sqrt(portvartotal))*100
# display(portfoliorisktotal)

In [ ]:
# Portfolio Sharpes calculations
portfoliosharpeexp = portfolioretexptotal/portfoliorisktotal
portfoliosharpe = portfoliorettotal/portfoliorisktotal
# display(portfoliosharpe)

In [ ]:
# Portfolio adding columns with Total Return, Total Risk and Sharpe values in first register(in the same line of IBOV value index). Other registers being filled with zero.
portfolio['RetETotal%'] = [portfolioretexptotal] + [0] * (len(portfolio) - 1)
portfolio['RetETotal%'] = portfolio['RetETotal%'].round(1)
portfolio['RetTotal%'] = [portfoliorettotal] + [0] * (len(portfolio) - 1)
portfolio['RetTotal%'] = portfolio['RetTotal%'].round(1)
portfolio['RiskTotal%'] = [portfoliorisktotal] + [0] * (len(portfolio) - 1)
portfolio['RiskTotal%'] = portfolio['RiskTotal%'].round(0)
portfolio['SharpeE'] = [portfoliosharpeexp] + [0] * (len(portfolio) - 1)
portfolio['SharpeE'] = portfolio['SharpeE'].round(3)
portfolio['Sharpe'] = [portfoliosharpe] + [0] * (len(portfolio) - 1)
portfolio['Sharpe'] = portfolio['Sharpe'].round(3)
# Calculate Total Beta and add in IBOV line
portfolio.at[0, 'Beta'] = (portfolio['W'] /100 * portfolio['Beta']).sum()
portfolio['Beta'] = portfolio['Beta'].round(3)
# display(portfolio)

In [ ]:
# Code structure begin to calculate sharpe maximizaton
#
# Funnctio to calculate negative Sharpe Ratio
def negative_sharpe_ratio(weights, expected_returns, cov_matrix, riskfree):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - riskfree) / portfolio_volatility
    return -sharpe_ratio  # negative to maximize in throught scipy.optimize minimize library

In [ ]:
# Extracting expected returns
expected_returns = portfolio['RetE%'].values

In [ ]:
# Convert covariance dataframe in numpy matrix
cov_matrix = covport.values

In [ ]:
weights = [values / 100 for values in weightport]

In [ ]:
# Weights restrictions = 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

In [ ]:
# Weights limits (no short opertions, only long ones)
bounds = tuple((0, 1) for _ in range(len(expected_returns)))

In [ ]:
# Initial weights to begin interaction
initial_weights = portfolio['W'].values

In [ ]:
# Optimization
result = minimize(
    negative_sharpe_ratio,
    initial_weights,
    args=(expected_returns, cov_matrix, riskfree),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

In [ ]:
# Final result
optimal_weights = result.x
max_sharpe = -result.fun  # negative once we multilpy to negative in the funcion operation
#
#  Code structure end to calculate sharpe maximizaton

In [ ]:
#  Add Maximum Sharpe ticker weights to portfolio dataframe
portfolio['SharpeEMax-W'] = optimal_weights
portfolio['SharpeEMax-W']  = ((portfolio['SharpeEMax-W'])*100).round(3)

In [ ]:
# Add Maximum Sharpe value to portfolio dataframe in IBOV line
portfolio.at[0, 'SharpeEMax-W'] = max_sharpe
# portfolio['SharpeEMax'] = [max_sharpe] + [0] * (len(portfolio) - 1)
portfolio['SharpeEMax-W'] = portfolio['SharpeEMax-W'].round(3)

In [ ]:
#  Add Maximum Sharpe ticker weights to portfolio dataframe
sharpeemaxret = (portfolio['RetE%'] * portfolio['SharpeEMax-W'] / 100).sum()
portfolio['SharpeEMax-Ret%'] = [sharpeemaxret]+ [0] * (len(portfolio) - 1)
portfolio['SharpeEMax-Ret%'] = portfolio['SharpeEMax-Ret%'].round(2)
display(portfolio)

,Ticker,W,RetE%,Ret%,Risk%,Beta,Min,Max,RetETotal%,RetTotal%,RiskTotal%,SharpeE,Sharpe,SharpeEMax-W,SharpeEMax-Ret%
0,IBOV,0.000,0.0,2.8,14.0,0.286,118533.00,137344.00,17.7,19.3,10.0,1.836,1.998,127.351,24.55
1,USDBRL,0.000,0.0,16.7,14.0,-0.004,4.97,6.30,0.0,0.0,0.0,0.000,0.000,0.000,0.00
2,BBAS3,5.885,27.3,7.9,19.0,0.810,23.44,29.08,0.0,0.0,0.0,0.000,0.000,31.761,0.00
3,CPFE3,3.688,15.0,14.0,20.0,0.602,31.00,39.15,0.0,0.0,0.0,0.000,0.000,0.000,0.00
4,ELET6,5.110,1.5,3.4,21.0,1.036,36.95,44.92,0.0,0.0,0.0,0.000,0.000,0.000,0.00
5,FBTC,3.314,30.0,53.7,59.0,0.444,260.22,573.73,0.0,0.0,0.0,0.000,0.000,1.997,0.00
6,HSML11,2.776,12.0,-9.5,16.0,0.098,68.09,90.83,0.0,0.0,0.0,0.000,0.000,0.000,0.00
7,HTMX11,6.154,32.4,0.4,24.0,-0.047,144.70,190.09,0.0,0.0,0.0,0.000,0.000,23.073,0.00
8,IVVB11,9.425,17.0,26.5,16.0,-0.239,284.40,417.20,0.0,0.0,0.0,0.000,0.000,16.843,0.00
9,PETR4,7.494,15.0,21.4,23.0,0.657,29.15,38.50,0.0,0.0,0.0,0.000,0.000,0.000,0.00


In [ ]:
# Autentication in Google Docs (only once)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# Open workbook and worksheets
wb = gc.open('Quotes')
wss = wb.worksheet('Stockvar')
wsr = wb.worksheet('RealStatevar')
wssus = wb.worksheet('StockUSvar')
wseus = wb.worksheet('ETFUSvar')
wsport = wb.worksheet('Portfolio')

In [ ]:
# Write data in the worksheets
wss.update([stockvar.columns.values.tolist()] + stockvar.values.tolist())
wsr.update([realstatevar.columns.values.tolist()] + realstatevar.values.tolist())
wssus.update([stockusvar.columns.values.tolist()] + stockusvar.values.tolist())
wseus.update([etfusvar.columns.values.tolist()] + etfusvar.values.tolist())
wsport.update([portfolio.columns.values.tolist()] + portfolio.values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'Portfolio!A1:O17',
 'updatedRows': 17,
 'updatedColumns': 15,
 'updatedCells': 255}